In [1]:
import argparse
import os
import warnings
import numpy as np
warnings.simplefilter('ignore', np.RankWarning)
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch.nn.functional as F
from depth_anything.dpt import LaplacianDepth
from util.loss import SiLogLoss
from dataset.custom_dataset import *

xFormers not available
xFormers not available


In [2]:
root_path = "../../autodl-tmp/deepthanything"
train_image_paths, train_depth_paths,val_image_paths, val_depth_paths = get_image_path(root_path)
print(len(train_image_paths),len(train_depth_paths),len(val_image_paths),len(val_depth_paths))

trans_t2_b includs image:103 and image:103 
trans_t2_a includs image:103 and image:103 
trans_t1_a includs image:164 and image:164 
trans_t2_c includs image:399 and image:399 
trans_t1_b includs image:1099 and image:1099 
879 879 220 220


In [3]:
import yaml
with open("config.yaml", "r") as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

parser = config["parser"]
model_configs = config["model_configs"]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args = argparse.Namespace(**parser)

In [4]:
trainset = CustomDataset(image_paths=train_image_paths, depth_paths=train_depth_paths, mode='train', size=(args.img_size, args.img_size))
trainloader = DataLoader(trainset, batch_size=args.bs, pin_memory=True, num_workers=4, drop_last=True, shuffle=True)
valset = CustomDataset(image_paths=val_image_paths, depth_paths=val_depth_paths, mode='val', size=(args.img_size, args.img_size))
valloader = DataLoader(valset, batch_size=args.bs, pin_memory=True, num_workers=4, drop_last=True, shuffle=True)

In [5]:
cudnn.enabled = True
cudnn.benchmark = True
model = LaplacianDepth(**{**model_configs['vitb'], 'max_depth': args.max_depth})
model.load_state_dict(torch.load(args.pretrained_from, map_location=device),strict=False)

/tmp/ipykernel_20147/1747136642.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.pretrained_from, map_location=device),strict=False)

_IncompatibleKeys(missing_keys=['depth_head.laplacian_filter.weight'], unexpected_keys=[])

In [6]:
model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
model.to(device)
criterion = SiLogLoss().to(device)
optimizer = AdamW([{'params': [param for name, param in model.named_parameters() if 'pretrained' in name], 'lr': args.lr},
                   {'params': [param for name, param in model.named_parameters() if 'pretrained' not in name], 'lr': args.lr * 10.0}],
                  lr=args.lr, betas=(0.9, 0.999), weight_decay=0.01)

previous_best = {'d1': 0, 'd2': 0, 'd3': 0, 'abs_rel': 100, 'sq_rel': 100, 'rmse': 100, 'rmse_log': 100, 'log10': 100, 'silog': 100}

In [ ]:
from train import *
running_loss = train(model,args,trainloader,valloader,optimizer,criterion,device)

Epoch 1/40: 100%|██████████| 219/219 [01:08<00:00,  3.19it/s]

Epoch [1/40] completed. Average Loss: 0.1571
